# Wrangling

In the following cells I perform the necessary steps in order to gather the data from multiple sources. There are three data sources that I will be collecting data from: 
 1. CSV that was handed to me
 2. Downloaded TSV file from online source
 3. JSON data from Twitter's API

In [1185]:
#importing all of the ncessary libraries
import pandas as pd
import requests
import json
import numpy as np
import tweepy
import os

### 1. Downloading and Loading the CSV File into a Dataframe
The CSV file was downloaded from Udactiy and stored on my local machine in the same folder location as my Jupyter Notebook. The file was then loaded into a dataframe using pandas as described in the cells below.

In [1186]:
#loading the archive file to a dataframe
df_archive = pd.read_csv('./twitter-archive-enhanced.csv')
df_archive.head() #verify the dataframe loads properly

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                                                               source  \
0  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
3  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
4  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

                                                                                                                                                                 text  \
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV   
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB   
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ   
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f   

   retweeted_status_id  retweeted_status_user_id retweeted_status_timestamp  \
0                  NaN                       NaN                        NaN   
1                  NaN                       NaN                        NaN   
2                  NaN                       NaN                        NaN   
3                  NaN                       NaN                        NaN   
4                  NaN                       NaN                        NaN   

                                                                                                                     expanded_urls  \
0                                                                  https://twitter.com/dog_rates/status/892420643555336193/photo/1   
1                                                                  https://twitter.com/dog_rates/status/892177421306343426/photo/1   
2                                                                  https://twitter.com/dog_rates/status/891815181378084864/photo/1   
3                                                                  https://twitter.com/dog_rates/status/891689557279858688/photo/1   
4  https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1   

   rating_numerator  rating_denominator      name doggo floofer pupper puppo  
0                13                  10   Phineas  None    None   None  None  
1                13                  10     Tilly  None    None   None  None  
2                12                  10    Archie  None    None   None  None  
3                13                  10     Darla  None    None   None  None  
4                12                  10  Franklin  None    None   None  None

### 2. Downloading the TSV File from the Internet
The next file needed to be downloaded programmatically from the internet using python. A URL was provided where I could download the file. Using the OS and Requests library in python, I was able to create a folder on my machine and make a request to the URL to download the file. 

After downloading the file, I was able to name the file based on the URL. Then, I stored the data onto a new dataframe using pandas.

In [1187]:
#created the folder to store the file that I needed to download
folder_name = 'image_predictions'
#create the folder only if the foldername does not exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)
#store the URL of the files location in url variable
url = ' https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
#store the request response in the response variable
response = requests.get(url)
#open the file path based on the foldername created in the first step
#create the file name based on the url, use all characters after the last '/', then write the file to the specified location
with open(os.path.join(folder_name,url.split('/')[-1]), mode = 'wb') as file:
    file.write(response.content)

In [1188]:
#since the file is a tsv open the file using pandas read_csv but use sep as \t for tabs and create the dataframe
df_image_pred = pd.read_csv(folder_name + '/' + url.split('/')[-1], sep='\t')
df_image_pred.head() #verify the dataframe created loads properly

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

### 3. Accessing JSON data from Twitter's API
The next data set would have to be accessed using Twitter's API. After following the instructions in the class, I was able to setup my own Twitter developer access and get my own authentication and token keys. I followed the instructions on how to access and get data from the Twitter API and used the Tweepy API class to get data. 

First, I needed to get the Tweet IDs from the Archive file (first file) that I loaded. After getting the Tweet IDs stored in a variable, I ran some simple tests to understand how the data was being returned using a single Tweet ID. I made sure I understood how to use the Tweepy API, get the JSON data, and download the JSON data for one Tweet ID before doing the entire list of tweet IDs. The rest of the steps I took are listed next to the cells below.

In [1189]:
#twitter API variables for authentication and access
#commenting out and removing keys and tokens
'''
consumer_key = 'xx'
consumer_secret = 'xx'
access_token = 'xx'
access_secret = 'xx'

#authentication based on twitter documentation
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
#use the tweepy API class to authenticate and also set wait on rate limit to True so that it waits when the limit is reached
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

'''

"\nconsumer_key = 'xx'\nconsumer_secret = 'xx'\naccess_token = 'xx'\naccess_secret = 'xx'\n\n#authentication based on twitter documentation\nauth = tweepy.OAuthHandler(consumer_key, consumer_secret)\nauth.set_access_token(access_token, access_secret)\n#use the tweepy API class to authenticate and also set wait on rate limit to True so that it waits when the limit is reached\napi = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)\n\n"

In [1190]:
#retweet count and favorite ("like") count at minimum,

In [1191]:
'''
#perform a test on one tweet ID to ensure that the connection is successful, use tweet_mode extended based on recommendation
tweet = api.get_status(892420643555336193, tweet_mode='extended')
#store the response as a json
tweet_json = tweet._json
#verify that I can read the json response as a dict using retweet_count as an example 
tweet_json['retweet_count']
#retweet_count
#favorite_count
'''

"\n#perform a test on one tweet ID to ensure that the connection is successful, use tweet_mode extended based on recommendation\ntweet = api.get_status(892420643555336193, tweet_mode='extended')\n#store the response as a json\ntweet_json = tweet._json\n#verify that I can read the json response as a dict using retweet_count as an example \ntweet_json['retweet_count']\n#retweet_count\n#favorite_count\n"

In [1192]:
#perform a simple test of creating a new file and writing all of the contents to the file
#commenting the below code out since I don't need it anymore
'''
with open('tweet_json.txt', 'w') as outfile:  
    json.dump(tweet_json, outfile)
'''

"\nwith open('tweet_json.txt', 'w') as outfile:  \n    json.dump(tweet_json, outfile)\n"

#### 3a. Downloading all of the JSON data into a text file
I stored all of the Tweet IDs from the archive data frame into a list which I would use when sending requests to the Twitter API. I created a simple script to help me loop through all of the Tweet IDs and store each JSON data for each Tweet in its own line in the text file. 

Because the script took a very long time to run (around 30 minutes), and since the script stores the text file on my local machine after it runs I didn't really need to run it again, I have commented out the script below. 

In [1193]:
tweet_id = df_archive['tweet_id'] #store all of the tweet IDs in df_archive into a list
tweet_errors = {} #creating a dictionary that will be used for the exceptions
''' 
### As stated in the above markdown cell, commenting out the rest of the download script as I like to rerun the entire the notebook and make it look clean ###

import time #importing the time library as suggessted to measure how long the script is running
start = time.time() #starting the timer
for tweetid in tweet_id: #starting the for loop for all Tweet IDs in the list created above
    try: #creating a try-except block to catch all of the exceptions as some tweets were deleted
        tweet = api.get_status(tweetid, tweet_mode='extended') #use the get_status from the Tweepy API to get the JSON data for each tweet ID
        tweet_json = tweet._json #convert the data into json--found the _json function online
        with open('tweet_json.txt', 'a') as outfile: #open a new file called tweet_json with the append action so that all of the data gets written to the file instead of overwriting it
            json.dump(tweet_json, outfile) #use the json.dump function to load the json data into the text file
            outfile.write("\n") #this creates a new line on the file for the next tweet
        end = time.time() #this stops the time for the last data that was appended to the file
        total_time = end-start #calculate the total time from start to finish of last load
        print(tweetid, (total_time/60)) #print out the tweet ID and the total time in minutes to see the progress
    except Exception as e: #for all exceptions store in this exception block
        print(str(tweetid) +": "+str(e)) #print the tweet ID along witht he exception error message
        tweet_errors[tweetid]=tweet_json #store the data in the tweet_errors dict that was created earlier
'''

' \n### As stated in the above markdown cell, commenting out the rest of the download script as I like to rerun the entire the notebook and make it look clean ###\n\nimport time #importing the time library as suggessted to measure how long the script is running\nstart = time.time() #starting the timer\nfor tweetid in tweet_id: #starting the for loop for all Tweet IDs in the list created above\n    try: #creating a try-except block to catch all of the exceptions as some tweets were deleted\n        tweet = api.get_status(tweetid, tweet_mode=\'extended\') #use the get_status from the Tweepy API to get the JSON data for each tweet ID\n        tweet_json = tweet._json #convert the data into json--found the _json function online\n        with open(\'tweet_json.txt\', \'a\') as outfile: #open a new file called tweet_json with the append action so that all of the data gets written to the file instead of overwriting it\n            json.dump(tweet_json, outfile) #use the json.dump function to 

#### 3b. Reading the text file line by line and loading the data into a dataframe
This next section took the longest time in the wrangling process. Multiple iterations were taken to get the data correctly and I had to look at the JSON data in the text file multiple times for various tweets to ensure that I was getting the data properly.

I created the below script to load the text file and then read each line and store the attributes that I wanted into a dataframe. The minimum required attributes were Tweet ID, Retweet Count, and Favorite Count. I had to look through the text file to get the actual attribute names in the JSON data using one Tweet ID as an example. 

I also decided that I wanted to pull Full Text, URL, and User Mentions just to see if I could. This would prove to be more difficult than I thought. The Full Text was easy to pull as it was just like the other minimum required fields. However, the URL and User Mentions were both nested in the JSON data tree. So, I had to go multiple layers to access both. 

I noticed that User Mentions were empty for a lot of data (more than 2,000) so I decided to remove that from the script. 

URL was nested for the majority of tweets in the same location. However, there were some exceptions that came up which is why I added the try-except blocks to the first part of the dataframe script. Some URLs were nested in different parts of the JSON data.

In [1194]:
#create a variable for the file that was created from step 3a with encoding set to utf-8
input_file = open('tweet_json.txt','r',encoding='utf-8')
df_list = [] #create a list that the attributes will be appended to
df_errors={} #create a dict for the exceptions
#start with the input_file variable
with input_file as f:
#start the for loop to go through each line of the input_file
    for line in f:
        try: #start the try-except block because of the exceptions
            data = json.loads(line) #use the json.load command to load the line as a json in the data variable
            tweet_id = data['id'] #start getting the attributes that I'm interested in starting with Tweet ID, since the data is in JSON format I can use the same functions as I would for a dict
            retweet_count = data['retweet_count'] #get the attribute retweet_count
            favorite_count = data['favorite_count'] #get the attribute favorite_count
            full_text = data['full_text'] #get the attribute full_text
            url = data['entities']['media'][0]['url'] #get the attribute url
            #append all of the attributes to df_list so that I can create a dataframe from the list
            df_list.append({'tweet_id':tweet_id, #create a key, value for tweet_id
                            'retweet_count':retweet_count, #create a key, value for retweet_count
                            'favorite_count':favorite_count, #create a key, value for favorite_count
                            'full_text':full_text, #create a key, value for full_text
                            'url':url, #create a key, value for url
                           })
            #use pandas DataFrame to convert the list into a dataframe with the appropriate columns
            df_tweepy = pd.DataFrame(df_list, columns = ['tweet_id',
                                                        'retweet_count',
                                                        'favorite_count',
                                                        'full_text',
                                                        'url'])
        #start the exception block as I noticed that some URLs are not stored in the same location of some tweets
        except Exception as e:
            print(str(tweet_id)+': '+str(e)) #print the exception message with the tweet ID
            df_errors[tweet_id]=line #store the line with the tweet ID into the dictionary
   

886267009285017600: 'media'
886054160059072513: 'media'
885518971528720385: 'media'
884247878851493888: 'media'
881633300179243008: 'media'
879674319642796034: 'media'
879130579576475649: 'media'
878604707211726852: 'media'
878404777348136964: 'media'
878316110768087041: 'media'
876537666061221889: 'media'
875097192612077568: 'media'
874434818259525634: 'media'
873337748698140672: 'media'
871166179821445120: 'media'
871102520638267392: 'media'
870726314365509632: 'media'
868639477480148993: 'media'
866720684873056260: 'media'
866094527597207552: 'media'
863471782782697472: 'media'
863427515083354112: 'media'
860981674716409858: 'media'
860177593139703809: 'media'
858860390427611136: 'media'
857214891891077121: 'media'
857062103051644929: 'media'
856602993587888130: 'media'
856330835276025856: 'media'
856288084350160898: 'media'
855862651834028034: 'media'
855860136149123072: 'media'
855857698524602368: 'media'
855818117272018944: 'media'
855245323840757760: 'media'
855138241867124737: 

In [1195]:
len(df_errors.keys()) #find the number of exceptions in the dictionary 

273

#### 3c. Creating a new Dataframe Ignoring Exceptions
Next, I wanted to create a new dataframe now that I have all of my exceptions. I wanted to get the list of all the tweet IDs that had the exception then run a modified version of the script I created in step 3b to avoid any exceptions. 

Getting the URLs for each of the exceptions proved to be challenging as the URL was nested in various places for these tweets. I didn't have enough time to go through each of the exceptions or write the code to handle all scenarios. So, if an exception existed, I just ignored the URL.

In [1196]:
tweet_errors = list(df_errors.keys()) #conver the Tweet IDs in the exception dict to a list
tweet_errors #verify the Tweet IDs

[886267009285017600,
 886054160059072513,
 885518971528720385,
 884247878851493888,
 881633300179243008,
 879674319642796034,
 879130579576475649,
 878604707211726852,
 878404777348136964,
 878316110768087041,
 876537666061221889,
 875097192612077568,
 874434818259525634,
 873337748698140672,
 871166179821445120,
 871102520638267392,
 870726314365509632,
 868639477480148993,
 866720684873056260,
 866094527597207552,
 863471782782697472,
 863427515083354112,
 860981674716409858,
 860177593139703809,
 858860390427611136,
 857214891891077121,
 857062103051644929,
 856602993587888130,
 856330835276025856,
 856288084350160898,
 855862651834028034,
 855860136149123072,
 855857698524602368,
 855818117272018944,
 855245323840757760,
 855138241867124737,
 852936405516943360,
 850333567704068097,
 849668094696017920,
 848213670039564288,
 847978865427394560,
 847617282490613760,
 846505985330044928,
 846139713627017216,
 845098359547420673,
 843981021012017153,
 841320156043304961,
 840761248237

In [1197]:
#clearing the list and datagrame since I am going to re-run a modified version of the script in the below cell
df_list = [] 
df_tweepy = []

In [1198]:
input_file = open('tweet_json.txt','r',encoding='utf-8') #store the file in a variable input_file
with input_file as f: #start by loading the file
    for line in f: #start the for loop
        data = json.loads(line) #use the json.load command to load the line as a json in the data variable
        tweet_id = data['id'] #start getting the attributes that I'm interested in starting with Tweet ID, since the data is in JSON format I can use the same functions as I would for a dict
        retweet_count = data['retweet_count'] #get the attribute retweet_count
        favorite_count = data['favorite_count'] #get the attribute favorite_count
        full_text = data['full_text'] #get the attribute full_text
        if  tweet_id not in tweet_errors: #if the Tweet ID does not exist in the Tweet ID exception list
            url = data['entities']['media'][0]['url'] #then use the following keys and store the value in the URL variable
        else: #if the Tweet ID does exist in the Tweet ID Exception list
            url=None #then don't put anything in the URL attribute
        #append all of the attributes to df_list so that I can create a dataframe from the list
        df_list.append({'tweet_id':tweet_id, #create a key, value for tweet_id
                        'retweet_count':retweet_count, #create a key, value for retweet_count
                        'favorite_count':favorite_count, #create a key, value for favorite_count
                        'full_text':full_text, #create a key, value for full_text
                        'url':url, #create a key, value for url
                        })
        #use pandas DataFrame to convert the list into a dataframe with the appropriate columns
        df_tweepy = pd.DataFrame(df_list, columns = ['tweet_id',
                                                    'retweet_count',
                                                    'favorite_count',
                                                    'full_text',
                                                    'url'])

In [1199]:
df_tweepy.count() #verify the counts, some URLs will be missing or null

tweet_id          2340
retweet_count     2340
favorite_count    2340
full_text         2340
url               2067
dtype: int64

# Assessing

Part of the assessment process was visually looking at all of the data set and then programmatically assessing the data to find data issues. In the cells below, I walk through all of the various data issues that I have identified and then I have aggregated all of my findings after my analysis.

The first step was to visually look at all of the data. I started with the archive table. I immediately noticed some tidiness issues with the dog classification (doggo, floofer, pupper, puppo). Separate attributes for these seemed unnecessary.

The rating number and rating denominator also seemed suspicious as it made no sense but that's just the way the twitter account rates the dogs so I left it as-is.

In [1200]:
df_archive

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0     892420643555336193                    NaN                  NaN   
1     892177421306343426                    NaN                  NaN   
2     891815181378084864                    NaN                  NaN   
3     891689557279858688                    NaN                  NaN   
4     891327558926688256                    NaN                  NaN   
5     891087950875897856                    NaN                  NaN   
6     890971913173991426                    NaN                  NaN   
7     890729181411237888                    NaN                  NaN   
8     890609185150312448                    NaN                  NaN   
9     890240255349198849                    NaN                  NaN   
10    890006608113172480                    NaN                  NaN   
11    889880896479866881                    NaN                  NaN   
12    889665388333682689                    NaN                  NaN   
13    889638837579907072                    NaN                  NaN   
14    889531135344209921                    NaN                  NaN   
15    889278841981685760                    NaN                  NaN   
16    888917238123831296                    NaN                  NaN   
17    888804989199671297                    NaN                  NaN   
18    888554962724278272                    NaN                  NaN   
19    888202515573088257                    NaN                  NaN   
20    888078434458587136                    NaN                  NaN   
21    887705289381826560                    NaN                  NaN   
22    887517139158093824                    NaN                  NaN   
23    887473957103951883                    NaN                  NaN   
24    887343217045368832                    NaN                  NaN   
25    887101392804085760                    NaN                  NaN   
26    886983233522544640                    NaN                  NaN   
27    886736880519319552                    NaN                  NaN   
28    886680336477933568                    NaN                  NaN   
29    886366144734445568                    NaN                  NaN   
...                  ...                    ...                  ...   
2326  666411507551481857                    NaN                  NaN   
2327  666407126856765440                    NaN                  NaN   
2328  666396247373291520                    NaN                  NaN   
2329  666373753744588802                    NaN                  NaN   
2330  666362758909284353                    NaN                  NaN   
2331  666353288456101888                    NaN                  NaN   
2332  666345417576210432                    NaN                  NaN   
2333  666337882303524864                    NaN                  NaN   
2334  666293911632134144                    NaN                  NaN   
2335  666287406224695296                    NaN                  NaN   
2336  666273097616637952                    NaN                  NaN   
2337  666268910803644416                    NaN                  NaN   
2338  666104133288665088                    NaN                  NaN   
2339  666102155909144576                    NaN                  NaN   
2340  666099513787052032                    NaN                  NaN   
2341  666094000022159362                    NaN                  NaN   
2342  666082916733198337                    NaN                  NaN   
2343  666073100786774016                    NaN                  NaN   
2344  666071193221509120                    NaN                  NaN   
2345  666063827256086533                    NaN                  NaN   
2346  666058600524156928                    NaN                  NaN   
2347  666057090499244032                    NaN                  NaN   
2348  666055525042405380                    NaN                  NaN   
2349  666051853826850816                    NaN         

Next I checked the data types for all of the columns. I noticed that the IDs are in floats and the timestamp was a string. These were some things that I could address as quality issues.

I also noticed a lot of None values in the classification was being counted as actual values.

In [1201]:
#check data types
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

I started looking through various attributes trying to see if there were any duplicated. I did notice that some of the expanded URLs were duplcated. These might be due to retweets.

In [1202]:
#check for duplicates
df_archive[df_archive['expanded_urls'].duplicated()]

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
55    881633300179243008           8.816070e+17         4.738443e+07   
64    879674319642796034           8.795538e+17         3.105441e+09   
75    878281511006478336                    NaN                  NaN   
76    878057613040115712                    NaN                  NaN   
98    873213775632977920                    NaN                  NaN   
113   870726314365509632           8.707262e+17         1.648776e+07   
126   868552278524837888                    NaN                  NaN   
135   866450705531457537                    NaN                  NaN   
136   866334964761202691                    NaN                  NaN   
148   863427515083354112           8.634256e+17         7.759620e+07   
150   863062471531167744                    NaN                  NaN   
161   860563773140209665                    NaN                  NaN   
169   859196978902773760                    NaN                  NaN   
179   857214891891077121           8.571567e+17         1.806710e+08   
185   856330835276025856                    NaN                  NaN   
186   856288084350160898           8.562860e+17         2.792810e+08   
188   855862651834028034           8.558616e+17         1.943518e+08   
189   855860136149123072           8.558585e+17         1.361572e+07   
218   850333567704068097           8.503288e+17         2.195506e+07   
228   848213670039564288           8.482121e+17         4.196984e+09   
234   847617282490613760           8.476062e+17         4.196984e+09   
253   844704788403113984                    NaN                  NaN   
264   842163532590374912                    NaN                  NaN   
271   841077006473256960                    NaN                  NaN   
274   840698636975636481           8.406983e+17         8.405479e+17   
276   840632337062862849                    NaN                  NaN   
280   839549326359670784                    NaN                  NaN   
290   838150277551247360           8.381455e+17         2.195506e+07   
291   838085839343206401           8.380855e+17         2.894131e+09   
293   837820167694528512                    NaN                  NaN   
...                  ...                    ...                  ...   
1598  686035780142297088           6.860340e+17         4.196984e+09   
1605  685681090388975616           6.855479e+17         4.196984e+09   
1618  684969860808454144           6.849598e+17         4.196984e+09   
1655  683391852557561860                    NaN                  NaN   
1663  682808988178739200           6.827884e+17         4.196984e+09   
1683  681694085539872773                    NaN                  NaN   
1688  681523177663676416                    NaN                  NaN   
1689  681340665377193984           6.813394e+17         4.196984e+09   
1726  680055455951884288                    NaN                  NaN   
1742  679462823135686656                    NaN                  NaN   
1774  678023323247357953           6.780211e+17         4.196984e+09   
1806  676936541936185344                    NaN                  NaN   
1819  676590572941893632           6.765883e+17         4.196984e+09   
1844  675849018447167488           6.758457e+17         4.196984e+09   
1895  674742531037511680           6.747400e+17         4.196984e+09   
1905  674606911342424069           6.744689e+17         4.196984e+09   
1914  674330906434379776           6.658147e+17         1.637468e+07   
1940  673716320723169284           6.737159e+17         4.196984e+09   
1970  673295268553605120                    NaN                  NaN   
2038  671550332464455680           6.715449e+17         4.196984e+09   
2149  669684865554620416           6.693544e+17         4.196984e+09   
2180  669000397445533696                    NaN                  NaN   
2189  668967877119254528           6.689207e+17         2.143566e+07   
2212  668587383441514497                    NaN         

Then I did a value count of all expanded URLs and noticed that most of the URLs were duplicated or some other websites other than twitter were referenced. 

In [1203]:
#check the value counts
df_archive['expanded_urls'].value_counts()

https://twitter.com/dog_rates/status/759923798737051648/photo/1                                                                                                                                                                                                                                        2
https://www.gofundme.com/my-puppys-double-cataract-surgery,https://twitter.com/dog_rates/status/825026590719483904/photo/1,https://twitter.com/dog_rates/status/825026590719483904/photo/1                                                                                                             2
https://twitter.com/dog_rates/status/753375668877008896/photo/1                                                                                                                                                                                                                                        2
https://twitter.com/dog_rates/status/667866724293877760/photo/1                                              

I used describe to check the min and max for the numerical values. I noticed that one denominator was 170 while all others were 10. So it might have been a typo. Then I checked all the values counts for denominator and found a small portion were not 10.

In [1204]:
#use describe to check the numerical values and see if they make sense
df_archive.describe()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
count  2.356000e+03           7.800000e+01         7.800000e+01   
mean   7.427716e+17           7.455079e+17         2.014171e+16   
std    6.856705e+16           7.582492e+16         1.252797e+17   
min    6.660209e+17           6.658147e+17         1.185634e+07   
25%    6.783989e+17           6.757419e+17         3.086374e+08   
50%    7.196279e+17           7.038708e+17         4.196984e+09   
75%    7.993373e+17           8.257804e+17         4.196984e+09   
max    8.924206e+17           8.862664e+17         8.405479e+17   

       retweeted_status_id  retweeted_status_user_id  rating_numerator  \
count         1.810000e+02              1.810000e+02       2356.000000   
mean          7.720400e+17              1.241698e+16         13.126486   
std           6.236928e+16              9.599254e+16         45.876648   
min           6.661041e+17              7.832140e+05          0.000000   
25%           7.186315e+17              4.196984e+09         10.000000   
50%           7.804657e+17              4.196984e+09         11.000000   
75%           8.203146e+17              4.196984e+09         12.000000   
max           8.874740e+17              7.874618e+17       1776.000000   

       rating_denominator  
count         2356.000000  
mean            10.455433  
std              6.745237  
min              0.000000  
25%             10.000000  
50%             10.000000  
75%             10.000000  
max            170.000000

In [1205]:
#check to see which denominators had 170
df_archive.query('rating_denominator ==170')['tweet_id']

1120    731156023742988288
Name: tweet_id, dtype: int64

In [1206]:
#check all value counts
df_archive_clean['rating_denominator'].value_counts()

10    2279
Name: rating_denominator, dtype: int64

Then I took a sample from df_archive to visually assess the data again. I noticed that the URLs was in the text field. 

In [1207]:
df_archive.sample(50)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
946   752568224206688256                    NaN                  NaN   
256   843981021012017153                    NaN                  NaN   
1766  678399652199309312                    NaN                  NaN   
1797  677269281705472000                    NaN                  NaN   
156   861383897657036800                    NaN                  NaN   
983   749395845976588288                    NaN                  NaN   
1958  673580926094458881                    NaN                  NaN   
192   855818117272018944                    NaN                  NaN   
2123  670385711116361728                    NaN                  NaN   
617   796387464403357696                    NaN                  NaN   
2349  666051853826850816                    NaN                  NaN   
2272  667491009379606528                    NaN                  NaN   
241   846505985330044928                    NaN                  NaN   
75    878281511006478336                    NaN                  NaN   
1802  676949632774234114                    NaN                  NaN   
434   820494788566847489                    NaN                  NaN   
369   828650029636317184                    NaN                  NaN   
1431  697516214579523584                    NaN                  NaN   
1840  675888385639251968                    NaN                  NaN   
2286  667182792070062081                    NaN                  NaN   
982   749403093750648834                    NaN                  NaN   
1313  706904523814649856                    NaN                  NaN   
2338  666104133288665088                    NaN                  NaN   
921   756288534030475264                    NaN                  NaN   
227   848324959059550208                    NaN                  NaN   
624   795464331001561088                    NaN                  NaN   
1100  735648611367784448                    NaN                  NaN   
1463  694669722378485760                    NaN                  NaN   
1108  734559631394082816                    NaN                  NaN   
1062  741099773336379392                    NaN                  NaN   
1873  675145476954566656                    NaN                  NaN   
1649  683742671509258241                    NaN                  NaN   
1216  714982300363173890                    NaN                  NaN   
2271  667495797102141441                    NaN                  NaN   
1212  715342466308784130                    NaN                  NaN   
92    874296783580663808                    NaN                  NaN   
37    885167619883638784                    NaN                  NaN   
1367  702598099714314240                    NaN                  NaN   
1480  693486665285931008                    NaN                  NaN   
485   814578408554463233                    NaN                  NaN   
351   831322785565769729                    NaN                  NaN   
2229  668248472370458624                    NaN                  NaN   
735   781163403222056960                    NaN                  NaN   
666   790581949425475584                    NaN                  NaN   
1606  685667379192414208                    NaN                  NaN   
1984  672902681409806336                    NaN                  NaN   
1522  690649993829576704                    NaN                  NaN   
2296  667090893657276420                    NaN                  NaN   
464   817415592588222464                    NaN                  NaN   
1441  696877980375769088                    NaN                  NaN   

                      timestamp  \
946   2016-07-11 18:20:21 +0000   
256   2017-03-21 00:22:10 +0000   
1766  2015-12-20 02:20:55 +0000   
1797  2015-12-16 23:29:14 +0000   
156   2017-05-08 00:54:59 +0000   
983   2016-07-03 00:14:27 +0000   
1958  2015-12-06 19:13:01 +0000   
192   2017-04-22 16:18:34 +0000   
2123  2015-11-27 23:36:23 +0

Next, I visually assessed the df_image_pred table to see if I noticed anything. I immediately noticed that there were multiple predictions for the same image. I also noticed that the puncutation of dog names were different.

In [1208]:
df_image_pred

tweet_id  \
0     666020888022790149   
1     666029285002620928   
2     666033412701032449   
3     666044226329800704   
4     666049248165822465   
5     666050758794694657   
6     666051853826850816   
7     666055525042405380   
8     666057090499244032   
9     666058600524156928   
10    666063827256086533   
11    666071193221509120   
12    666073100786774016   
13    666082916733198337   
14    666094000022159362   
15    666099513787052032   
16    666102155909144576   
17    666104133288665088   
18    666268910803644416   
19    666273097616637952   
20    666287406224695296   
21    666293911632134144   
22    666337882303524864   
23    666345417576210432   
24    666353288456101888   
25    666362758909284353   
26    666373753744588802   
27    666396247373291520   
28    666407126856765440   
29    666411507551481857   
...                  ...   
2045  886366144734445568   
2046  886680336477933568   
2047  886736880519319552   
2048  886983233522544640   
2049  887101392804085760   
2050  887343217045368832   
2051  887473957103951883   
2052  887517139158093824   
2053  887705289381826560   
2054  888078434458587136   
2055  888202515573088257   
2056  888554962724278272   
2057  888804989199671297   
2058  888917238123831296   
2059  889278841981685760   
2060  889531135344209921   
2061  889638837579907072   
2062  889665388333682689   
2063  889880896479866881   
2064  890006608113172480   
2065  890240255349198849   
2066  890609185150312448   
2067  890729181411237888   
2068  890971913173991426   
2069  891087950875897856   
2070  891327558926688256   
2071  891689557279858688   
2072  891815181378084864   
2073  892177421306343426   
2074  892420643555336193   

                                                                                      jpg_url  \
0                                             https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1                                             https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2                                             https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3                                             https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4                                             https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   
5                                             https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg   
6                                             https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg   
7                                             https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg   
8                                             https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg   
9                                             https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg   
10                                            https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg   
11                                            https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg   
12                                            https://pbs.twimg.com/media/CT5d9DZXAAALcwe.jpg   
13                                            https://pbs.twimg.com/media/CT5m4VGWEAAtKc8.jpg   
14                                            https://pbs.twimg.com/media/CT5w9gUW4AAsBNN.jpg   
15                                            https://pbs.twimg.com/media/CT51-JJUEAA6hV8.jpg   
16                                            https://pbs.twimg.com/media/CT54YGiWUAEZnoK.jpg   
17                                            https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg   
18                                            https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg   
19                                            https://pbs.twimg.com/media/CT8T1mtUwAA3aqm.jpg   
20                                            https://pbs.twimg.com/media/CT8g3BpUEAAuFjg.jpg   
21                                            https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg   
22                                            ht

I checked the data types for the table as well and everything seemed okay.

In [1209]:
#check the data types
df_image_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


I checked for duplicated data of the image to see if some predictions were duplicates. Turns out that the same image was predicted multiple times. This was probably due to the retweets.

In [1210]:
#checjk duplicates for images
df_image_pred[df_image_pred['jpg_url'].duplicated()]

tweet_id  \
1297  752309394570878976   
1315  754874841593970688   
1333  757729163776290825   
1345  759159934323924993   
1349  759566828574212096   
1364  761371037149827077   
1368  761750502866649088   
1387  766078092750233600   
1407  770093767776997377   
1417  771171053431250945   
1427  772615324260794368   
1446  775898661951791106   
1453  776819012571455488   
1456  777641927919427584   
1463  778396591732486144   
1476  780496263422808064   
1487  782021823840026624   
1495  783347506784731136   
1510  786036967502913536   
1522  788070120937619456   
1538  790723298204217344   
1541  791026214425268224   
1564  793614319594401792   
1569  794355576146903043   
1571  794983741416415232   
1579  796177847564038144   
1588  798340744599797760   
1589  798628517273620480   
1590  798644042770751489   
1591  798665375516884993   
...                  ...   
1619  802624713319034886   
1624  803692223237865472   
1627  804413760345620481   
1634  805958939288408065   
1636  806242860592926720   
1640  807059379405148160   
1645  808134635716833280   
1652  809808892968534016   
1683  813944609378369540   
1693  816014286006976512   
1699  816829038950027264   
1703  817181837579653120   
1712  818588835076603904   
1717  819015331746349057   
1718  819015337530290176   
1727  820446719150292993   
1736  821813639212650496   
1742  822647212903690241   
1746  823269594223824897   
1755  824796380199809024   
1789  829878982036299777   
1803  832040443403784192   
1804  832215726631055365   
1858  841833993020538882   
1864  842892208864923648   
1903  851953902622658560   
1944  861769973181624320   
1992  873697596434513921   
2041  885311592912609280   
2055  888202515573088257   

                                                                                      jpg_url  \
1297  https://pbs.twimg.com/ext_tw_video_thumb/675354114423808004/pu/img/qL1R_nGLqa6lmkOx.jpg   
1315                                          https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg   
1333                                          https://pbs.twimg.com/media/CWyD2HGUYAQ1Xa7.jpg   
1345                                          https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg   
1349                                          https://pbs.twimg.com/media/CkNjahBXAAQ2kWo.jpg   
1364                              https://pbs.twimg.com/tweet_video_thumb/CeBym7oXEAEWbEg.jpg   
1368                                          https://pbs.twimg.com/media/CYLDikFWEAAIy1y.jpg   
1387                                          https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg   
1407                                          https://pbs.twimg.com/media/CkjMx99UoAM2B1a.jpg   
1417                                          https://pbs.twimg.com/media/CVgdFjNWEAAxmbq.jpg   
1427                                          https://pbs.twimg.com/media/Cp6db4-XYAAMmqL.jpg   
1446                                          https://pbs.twimg.com/media/CiyHLocU4AI2pJu.jpg   
1453                                          https://pbs.twimg.com/media/CW88XN4WsAAlo8r.jpg   
1456                                          https://pbs.twimg.com/media/CmoPdmHW8AAi8BI.jpg   
1463                                          https://pbs.twimg.com/media/CcG07BYW0AErrC9.jpg   
1476                                          https://pbs.twimg.com/media/Ck2d7tJWUAEPTL3.jpg   
1487                                          https://pbs.twimg.com/media/CdHwZd0VIAA4792.jpg   
1495                                          https://pbs.twimg.com/media/CVuQ2LeUsAAIe3s.jpg   
1510                                          https://pbs.twimg.com/media/CtKHLuCWYAA2TTs.jpg   
1522                                          https://pbs.twimg.com/media/Co-hmcYXYAASkiG.jpg   
1538                                          https://pbs.twimg.com/media/CvaYgDOWgAEfjls.jpg   
1541                                          https://pbs.twimg.com/media/CpmyNumW8AAAJGj.jpg   
1564                                          ht

In [1211]:
#check for a specific image url using query to check the duplicate
df_image_pred.query('jpg_url =="https://pbs.twimg.com/media/C4KHj-nWQAA3poV.jpg"')

tweet_id                                          jpg_url  \
1785  829374341691346946  https://pbs.twimg.com/media/C4KHj-nWQAA3poV.jpg   
1903  851953902622658560  https://pbs.twimg.com/media/C4KHj-nWQAA3poV.jpg   

      img_num                         p1   p1_conf  p1_dog  \
1785        1  Staffordshire_bullterrier  0.757547    True   
1903        1  Staffordshire_bullterrier  0.757547    True   

                                  p2  p2_conf  p2_dog  \
1785  American_Staffordshire_terrier  0.14995    True   
1903  American_Staffordshire_terrier  0.14995    True   

                            p3   p3_conf  p3_dog  
1785  Chesapeake_Bay_retriever  0.047523    True  
1903  Chesapeake_Bay_retriever  0.047523    True

I used describe to see if any of the numerical values were suspect.

In [1212]:
df_image_pred.describe()

tweet_id      img_num      p1_conf       p2_conf       p3_conf
count  2.075000e+03  2075.000000  2075.000000  2.075000e+03  2.075000e+03
mean   7.384514e+17     1.203855     0.594548  1.345886e-01  6.032417e-02
std    6.785203e+16     0.561875     0.271174  1.006657e-01  5.090593e-02
min    6.660209e+17     1.000000     0.044333  1.011300e-08  1.740170e-10
25%    6.764835e+17     1.000000     0.364412  5.388625e-02  1.622240e-02
50%    7.119988e+17     1.000000     0.588230  1.181810e-01  4.944380e-02
75%    7.932034e+17     1.000000     0.843855  1.955655e-01  9.180755e-02
max    8.924206e+17     4.000000     1.000000  4.880140e-01  2.734190e-01

In [1213]:
#performed value count on dogs to see if there were any duplicates based on mis-punctuation
df_image_pred['p3'].value_counts()

Labrador_retriever                79
Chihuahua                         58
golden_retriever                  48
Eskimo_dog                        38
kelpie                            35
kuvasz                            34
Staffordshire_bullterrier         32
chow                              32
cocker_spaniel                    31
beagle                            31
toy_poodle                        29
Pomeranian                        29
Pekinese                          29
Great_Pyrenees                    27
Chesapeake_Bay_retriever          27
Pembroke                          27
French_bulldog                    26
malamute                          26
American_Staffordshire_terrier    24
Cardigan                          23
pug                               23
basenji                           21
toy_terrier                       20
bull_mastiff                      20
Siberian_husky                    19
Shetland_sheepdog                 17
Boston_bull                       17
L

The last table I assessed was the df_tweepy table to see if there were any quality or tidiness issues. Only two new attributes were present on this dataset and the others seemed like duplicate. This table seemed like a good candidate to merge with another table.

In [1214]:
df_tweepy

tweet_id  retweet_count  favorite_count  \
0     892420643555336193           8311           38002   
1     892177421306343426           6139           32632   
2     891815181378084864           4064           24549   
3     891689557279858688           8447           41351   
4     891327558926688256           9154           39532   
5     891087950875897856           3043           19862   
6     890971913173991426           2018           11606   
7     890729181411237888          18435           64126   
8     890609185150312448           4182           27277   
9     890240255349198849           7214           31302   
10    890006608113172480           7170           30080   
11    889880896479866881           4864           27273   
12    889665388333682689           9819           47193   
13    889638837579907072           4433           26633   
14    889531135344209921           2198           14830   
15    889278841981685760           5253           24766   
16    888917238123831296           4400           28563   
17    888804989199671297           4192           25075   
18    888554962724278272           3471           19441   
19    888078434458587136           3415           21336   
20    887705289381826560           5275           29601   
21    887517139158093824          11420           45398   
22    887473957103951883          17766           67726   
23    887343217045368832          10171           33043   
24    887101392804085760           5832           29990   
25    886983233522544640           7602           34473   
26    886736880519319552           3209           11833   
27    886680336477933568           4363           21997   
28    886366144734445568           3129           20787   
29    886267009285017600              4             116   
...                  ...            ...             ...   
2310  666411507551481857            323             438   
2311  666407126856765440             40             105   
2312  666396247373291520             84             164   
2313  666373753744588802             90             185   
2314  666362758909284353            562             763   
2315  666353288456101888             71             215   
2316  666345417576210432            133             288   
2317  666337882303524864             90             193   
2318  666293911632134144            347             491   
2319  666287406224695296             64             143   
2320  666273097616637952             76             171   
2321  666268910803644416             35             100   
2322  666104133288665088           6430           14074   
2323  666102155909144576             12              78   
2324  666099513787052032             66             150   
2325  666094000022159362             72             161   
2326  666082916733198337             44             114   
2327  666073100786774016            160             316   
2328  666071193221509120             59             142   
2329  666063827256086533            213             465   
2330  666058600524156928             57             109   
2331  666057090499244032            139             290   
2332  666055525042405380            237             429   
2333  666051853826850816            837            1204   
2334  666050758794694657             59             130   
2335  666049248165822465             41             106   
2336  666044226329800704            139             293   
2337  666033412701032449             43             123   
2338  666029285002620928             47             126   
2339  666020888022790149            501            2531   

                                                                                                                                                                   full_text  \
0                                                              This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU   
1    

In [1215]:
#check the data types
df_tweepy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2340 entries, 0 to 2339
Data columns (total 5 columns):
tweet_id          2340 non-null int64
retweet_count     2340 non-null int64
favorite_count    2340 non-null int64
full_text         2340 non-null object
url               2067 non-null object
dtypes: int64(3), object(2)
memory usage: 91.5+ KB


In [1216]:
#check the numericalvalues distribution
df_tweepy.describe()

tweet_id  retweet_count  favorite_count
count  2.340000e+03    2340.000000     2340.000000
mean   7.422176e+17    2926.808120     7955.481624
std    6.832564e+16    4930.476731    12321.539910
min    6.660209e+17       0.000000        0.000000
25%    6.783394e+17     587.750000     1373.000000
50%    7.186224e+17    1366.500000     3458.500000
75%    7.986954e+17    3410.250000     9734.000000
max    8.924206e+17   83567.000000   164162.000000

In [1217]:
#check the favorite counts
df_tweepy.query('favorite_count == 0')

tweet_id  retweet_count  favorite_count  \
31    886054160059072513            105               0   
35    885311592912609280          18153               0   
67    879130579576475649           6694               0   
72    878404777348136964           1267               0   
73    878316110768087041           6525               0   
77    877611172832227328             79               0   
90    874434818259525634          14496               0   
95    873337748698140672           1564               0   
106   871166179821445120           5662               0   
120   868639477480148993           2097               0   
126   867072653475098625            121               0   
132   866094527597207552           8406               0   
141   863471782782697472           2476               0   
153   860981674716409858           2187               0   
154   860924035999428608            835               0   
159   860177593139703809          31380               0   
165   858860390427611136           8315               0   
174   857062103051644929            177               0   
176   856602993587888130          10965               0   
179   856330835276025856            699               0   
188   855245323840757760           6177               0   
189   855138241867124737             47               0   
198   852936405516943360           2089               0   
205   851953902622658560          10064               0   
206   851861385021730816             23               0   
216   849668094696017920           5208               0   
224   847978865427394560           3436               0   
225   847971574464610304            449               0   
243   845098359547420673           8113               0   
258   841833993020538882          16323               0   
...                  ...            ...             ...   
761   776249906839351296           9947               0   
766   775898661951791106          17876               0   
781   773336787167145985           5533               0   
787   772615324260794368           3665               0   
798   771171053431250945           8203               0   
802   771004394259247104            238               0   
808   770093767776997377           3295               0   
812   769335591808995329           8336               0   
815   768909767477751808           2920               0   
819   768554158521745409           6312               0   
827   766864461642756096           6119               0   
833   766078092750233600           2816               0   
846   763167063695355904           3262               0   
854   761750502866649088           4253               0   
858   761371037149827077          19410               0   
871   760153949710192640             34               0   
876   759566828574212096          22799               0   
881   759159934323924993           1253               0   
894   757729163776290825           8741               0   
897   757597904299253760            314               0   
912   754874841593970688           8622               0   
922   753298634498793472           6205               0   
928   752701944171524096           3114               0   
934   752309394570878976          17974               0   
997   747242308580548608           3083               0   
1008  746521445350707200           1052               0   
1028  743835915802583040           2229               0   
1227  711998809858043904            133               0   
2243  667550904950915073             35               0   
2244  667550882905632768             33               0   

                                                                                                                                           full_text  \
31                                                                                                RT @Athletics: 12/10 #BATP https://t.co/WxwJmvjfxo   
35             RT @dog_rates: This is Lilly. She just

# Cleaning

In [1218]:
# copy all of the dataframes for cleaning activities
df_archive_clean = df_archive.copy()
df_tweepy_clean = df_tweepy.copy()
df_image_pred_clean = df_image_pred.copy()

## Quality Issues

#### df_archive table
1. Dog names don't seem to be correct (visually noticed 'the' and 'a' as some dog names)
2. Timestamp and retweeted_status_timestamp is represented as an object and should be converted to a datetime.
3. Expanded URLs seem to consist of duplicate Twitter URLs and also URLs to other sites (like go fund me) all jumbled into a single attribute
4. None values are being counted when they should be NaN
5. Small portion of rating denominators are not 10 and are inconsistent.
6. Retweets seem like duplicate data in both archive and image_pred tables
7. IDs should be integers instead of float.

#### df_image_pred table
8. Inconsistent punctuation of the dog types in the p1, p2, and p3 columns.

## Tidyness Issues

#### df_archive table
1. Multiple attributes represented when it can be combined for dog type (i.e. doggo, floofer, pupper, puppo) even rating (numerator and denominator)
2. URLs are in the text attributes

#### df_tweepy table
3. Only has a two attributes that are new (retweet count and favorite count). This should be merged to the df_archive table
4. Multiple attributes on the table that are very similar with the prediction confidence score getting worse with more predictions (i.e. prediction, prediction number, and dog (true or false))

My approach was to handle all of the tidyness issues first. Tidyness issue 1 and Quality issue 4 needed to be cleaned up together to make things easier.

### Tidyness Issue 1 & Quality Issue 4
I'll handle both of these data issues first.
#### df_archive table
- Multiple attributes represented when it can be combined for dog type (i.e. doggo, floofer, pupper, puppo) even rating (numerator and denominator)
- None values are being counted when they should be NaN


##### Define
- Combine the attributes (doggo, floofer, puuper, puppo) into a single value called classification.
- Replace the None values with NaN so they are not counted as values

##### Code

In [1219]:
#Replace the None value with empty character. Create a variable for the column names with None as a value
ids =['name','doggo','floofer','pupper','puppo']

#create a function to replace the None value with an empty character
def replaceNone(df, id):
    df[id] = df[id].replace('None','')
#use the function for the column names on the dataframe
replaceNone(df_archive_clean, ids)
#merge the doggo, floofer, pupper, and puppo  column values
df_archive_clean['classification']=df_archive_clean['doggo']+df_archive_clean['floofer']+df_archive_clean['pupper']+df_archive_clean['puppo']
#drop the merged columns
df_archive_clean=df_archive_clean.drop(['doggo','floofer','pupper','puppo'],axis=1)
#replace the empty characters with np.nan so it doesn't get counted
df_archive_clean['name'].replace('',np.nan,inplace=True)
#replace the empty characters with np.nan so it doesn't get counted
df_archive_clean['classification'].replace('',np.nan,inplace=True)

In [1220]:
#change datatype to category
df_archive_clean['classification'] = df_archive_clean['classification'].astype('category')

##### Test

In [1221]:
#take a sample of the dataframe to ensure all of the columns were merged
df_archive_clean.sample(50)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
2183  668989615043424256                    NaN                  NaN   
1106  734787690684657664                    NaN                  NaN   
498   813130366689148928           8.131273e+17         4.196984e+09   
1661  683030066213818368                    NaN                  NaN   
2297  667073648344346624                    NaN                  NaN   
1514  691090071332753408                    NaN                  NaN   
620   796125600683540480                    NaN                  NaN   
663   790946055508652032                    NaN                  NaN   
501   813096984823349248                    NaN                  NaN   
366   828801551087042563                    NaN                  NaN   
1028  745789745784041472                    NaN                  NaN   
275   840696689258311684                    NaN                  NaN   
471   816697700272001025                    NaN                  NaN   
1714  680440374763077632                    NaN                  NaN   
1665  682750546109968385                    NaN                  NaN   
962   750868782890057730                    NaN                  NaN   
2097  670733412878163972                    NaN                  NaN   
613   796865951799083009                    NaN                  NaN   
290   838150277551247360           8.381455e+17         2.195506e+07   
2106  670468609693655041                    NaN                  NaN   
1422  698178924120031232                    NaN                  NaN   
2015  672205392827572224                    NaN                  NaN   
25    887101392804085760                    NaN                  NaN   
1732  679828447187857408                    NaN                  NaN   
1901  674646392044941312                    NaN                  NaN   
1479  693582294167244802           6.935722e+17         1.198989e+09   
1529  690248561355657216                    NaN                  NaN   
2323  666428276349472768                    NaN                  NaN   
1892  674754018082705410           6.747522e+17         4.196984e+09   
1244  711743778164514816                    NaN                  NaN   
378   827933404142436356                    NaN                  NaN   
1829  676237365392908289                    NaN                  NaN   
192   855818117272018944                    NaN                  NaN   
1873  675145476954566656                    NaN                  NaN   
877   760893934457552897                    NaN                  NaN   
293   837820167694528512                    NaN                  NaN   
1920  674265582246694913                    NaN                  NaN   
699   786286427768250368                    NaN                  NaN   
671   789960241177853952                    NaN                  NaN   
1088  737826014890496000                    NaN                  NaN   
998   748307329658011649                    NaN                  NaN   
897   759047813560868866                    NaN                  NaN   
756   778650543019483137                    NaN                  NaN   
42    884247878851493888                    NaN                  NaN   
951   751950017322246144                    NaN                  NaN   
2032  671763349865160704                    NaN                  NaN   
1093  737310737551491075                    NaN                  NaN   
2328  666396247373291520                    NaN                  NaN   
0     892420643555336193                    NaN                  NaN   
908   757729163776290825                    NaN                  NaN   

                      timestamp  \
2183  2015-11-24 03:08:48 +0000   
1106  2016-05-23 16:46:51 +0000   
498   2016-12-25 21:12:41 +0000   
1661  2016-01-01 21:00:32 +0000   
2297  2015-11-18 20:15:26 +0000   
1514  2016-01-24 02:48:07 +0000   
620   2016-11-08 23:01:49 +0000   
663   2016-10-25 16:00:09 +0000   
501   2016-12-25 19:00:02 +0

In [1222]:
#check a specific tweet value against the old archive value
df_archive_clean.query('tweet_id ==803773340896923648')['classification'], df_archive.query('tweet_id ==803773340896923648')['puppo'] 

(554    puppo
 Name: classification, dtype: category
 Categories (7, object): [doggo, doggofloofer, doggopupper, doggopuppo, floofer, pupper, puppo],
 554    puppo
 Name: puppo, dtype: object)

### Tidyness Issue 2
#### df_archive table
- URLs are in the text attribute

##### Define
- Extract the only the text from text attribute and since a URL attribute already exists we don't need to extract the URL contained in the text attribute

##### Code

In [1223]:
#extract all characters using regex before the http and only look at text fields that have an http. store the clean values in a column text_clean
df_archive_clean['text_clean'] = df_archive_clean[df_archive_clean['text'].str.contains('http')].text.str.extract('(.[\s\S]+?(?=https))', expand=True) 

#fill all of the null values in text_clean with the values that didn't have http in the original text column
df_archive_clean['text_clean'].fillna(df_archive_clean['text'], inplace=True)

##### Test

In [1224]:
#check some tweets to ensure the link is removed
df_archive_clean.query('tweet_id in (679158373988876288, 754874841593970688)')['text_clean']

926     RT @dog_rates: This is Rubio. He has too much skin. 11/10 
1744                   This is Rubio. He has too much skin. 11/10 
Name: text_clean, dtype: object

In [1225]:
#check the number of rows
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 15 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          1611 non-null object
classification                380 non-null category
text_clean                    2356 non-null object
dtypes: category(1), float64(4), int64(3), object(7)
memory usage: 260.4+ KB


In [1226]:
#check to see if any string contains http
df_archive_clean[df_archive_clean['text_clean'].str.contains('http')]

Empty DataFrame
Columns: [tweet_id, in_reply_to_status_id, in_reply_to_user_id, timestamp, source, text, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls, rating_numerator, rating_denominator, name, classification, text_clean]
Index: []

In [1227]:
#dropping the text column after performing the tests since it is no longer required
df_archive_clean.drop('text',axis=1,inplace=True)

In [1228]:
#checking to make sure the column was dropped
df_archive_clean.head()

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   
3  891689557279858688                    NaN                  NaN   
4  891327558926688256                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   
3  2017-07-30 15:58:51 +0000   
4  2017-07-29 16:00:24 +0000   

                                                                               source  \
0  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
2  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
3  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
4  <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   

   retweeted_status_id  retweeted_status_user_id retweeted_status_timestamp  \
0                  NaN                       NaN                        NaN   
1                  NaN                       NaN                        NaN   
2                  NaN                       NaN                        NaN   
3                  NaN                       NaN                        NaN   
4                  NaN                       NaN                        NaN   

                                                                                                                     expanded_urls  \
0                                                                  https://twitter.com/dog_rates/status/892420643555336193/photo/1   
1                                                                  https://twitter.com/dog_rates/status/892177421306343426/photo/1   
2                                                                  https://twitter.com/dog_rates/status/891815181378084864/photo/1   
3                                                                  https://twitter.com/dog_rates/status/891689557279858688/photo/1   
4  https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1   

   rating_numerator  rating_denominator      name classification  \
0                13                  10   Phineas            NaN   
1                13                  10     Tilly            NaN   
2                12                  10    Archie            NaN   
3                13                  10     Darla            NaN   
4                12                  10  Franklin            NaN   

                                                                                                                                    text_clean  
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10   
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10   
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10   
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us   
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek

### Tidyness Issue 3
#### df_tweepy table
- Only has a two attributes that are new (retweet count and favorite count). This should be merged to the df_archive table

##### Define
- Merge with the df_archive_clean table

##### Code

In [1229]:
#merge archive and tweepy dataframes
df_archive_clean = df_archive_clean.merge(df_tweepy_clean, on='tweet_id')

In [1230]:
#drop full_text columns as it is a duplicate. URL is also a duplicate but I want to keep the shortened version
df_archive_clean.drop('full_text', axis=1,inplace=True)

##### Test

In [1231]:
#verify a with a sample set of data that the data has been merged
df_archive_clean.sample(50)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
924   753039830821511168                    NaN                  NaN   
1729  679148763231985668                    NaN                  NaN   
442   818614493328580609                    NaN                  NaN   
1153  721001180231503872                    NaN                  NaN   
274   839239871831150596                    NaN                  NaN   
1478  692752401762250755                    NaN                  NaN   
1764  677700003327029250                    NaN                  NaN   
1591  685667379192414208                    NaN                  NaN   
2161  669037058363662336                    NaN                  NaN   
2196  668587383441514497                    NaN                  NaN   
1360  701889187134500865                    NaN                  NaN   
609   796116448414461957                    NaN                  NaN   
1583  686035780142297088           6.860340e+17         4.196984e+09   
347   830583320585068544                    NaN                  NaN   
2200  668537837512433665                    NaN                  NaN   
2268  667192066997374976                    NaN                  NaN   
1199  715200624753819648                    NaN                  NaN   
1882  674690135443775488                    NaN                  NaN   
1101  732585889486888962                    NaN                  NaN   
2053  671134062904504320                    NaN                  NaN   
1699  680440374763077632                    NaN                  NaN   
497   812503143955202048                    NaN                  NaN   
2248  667538891197542400                    NaN                  NaN   
624   793500921481273345                    NaN                  NaN   
1410  697943111201378304                    NaN                  NaN   
176   856602993587888130                    NaN                  NaN   
1256  709409458133323776                    NaN                  NaN   
621   793845145112371200                    NaN                  NaN   
2047  671159727754231808                    NaN                  NaN   
946   751132876104687617                    NaN                  NaN   
453   817423860136083457                    NaN                  NaN   
471   815639385530101762                    NaN                  NaN   
388   825026590719483904                    NaN                  NaN   
133   865718153858494464                    NaN                  NaN   
1000  747103485104099331                    NaN                  NaN   
1844  675489971617296384                    NaN                  NaN   
1144  724046343203856385                    NaN                  NaN   
1566  687124485711986689                    NaN                  NaN   
92    874057562936811520                    NaN                  NaN   
462   816450570814898180                    NaN                  NaN   
35    885311592912609280                    NaN                  NaN   
1461  693629975228977152                    NaN                  NaN   
94    873580283840344065                    NaN                  NaN   
377   826598799820865537           8.265984e+17         4.196984e+09   
2261  667437278097252352                    NaN                  NaN   
775   774314403806253056                    NaN                  NaN   
2080  670755717859713024                    NaN                  NaN   
549   802952499103731712                    NaN                  NaN   
1955  673270968295534593                    NaN                  NaN   
62    879862464715927552                    NaN                  NaN   

                      timestamp  \
924   2016-07-13 01:34:21 +0000   
1729  2015-12-22 03:57:37 +0000   
442   2017-01-10 00:24:38 +0000   
1153  2016-04-15 15:44:11 +0000   
274   2017-03-07 22:22:32 +0000   
1478  2016-01-28 16:53:37 +0000   
1764  2015-12-18 04:00:46 +0000   
1591  2016-01-09 03:40:16 +0000   
2161  2015-11-24 06:17:19 +0

### Tidyness Issue 4
#### df_image_pred table 
- Multiple attributes on the table that are very similar with the prediction confidence score getting worse with more predictions (i.e. prediction, prediction number, and dog (true or false))

##### Define
- Drop the columns for predictions 2 and predictions 3 and rename prediction 1 columns

##### Code

In [1232]:
#create a variable to drop column headers 
drop_columns=['p2','p2_conf','p2_dog','p3','p3_conf','p3_dog']
#drop the column headers from the dataframe
df_image_pred_clean = df_image_pred_clean.drop(columns=drop_columns, axis=1)

In [1233]:
#rename the columns
df_image_pred_clean.rename(columns={'p1':'prediction','p1_conf':'confidence','p1_dog':'dog'},inplace=True)

##### Test

In [1234]:
#verify the columns are renamed and dropped
df_image_pred_clean.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num              prediction  confidence   dog  
0        1  Welsh_springer_spaniel    0.465074  True  
1        1                 redbone    0.506826  True  
2        1         German_shepherd    0.596461  True  
3        1     Rhodesian_ridgeback    0.408143  True  
4        1      miniature_pinscher    0.560311  True

### Quality Issues

#### df_archive table
1. Dog names don't seem to be correct (visually noticed 'the' and 'a' as some dog names)
2. Timestamp and retweeted_status_timestamp is represented as an object and should be converted to a datetime.
3. Expanded URLs seem to consist of duplicate Twitter URLs and also URLs to other sites (like go fund me) all jumbled into a single attribute
4. None values are being counted when they should be NaN
5. URLs are in the text attributes (tidiness issue)
6. Retweets seem like duplicate data in both archive and image_pred tables
7. IDs should be integers instead of float.

#### df_image_pred table
8. Inconsistent punctuation of the dog types in the p1, p2, and p3 columns.

### Quality Issue 1
Dog names don't seem to be correct (visually noticed 'the' and 'a' as some dog names)

##### Define
- Remove invalid names (a, an, the) and see if there is a way to extract the name

##### Code

In [1235]:
#create a variable to store the invalid names identified
invalid_names = ['a','an','the']
#replace the invalid names with NaN
df_archive_clean['name'].replace(invalid_names, np.nan, inplace=True)

##### Test

In [1236]:
#check to make sure the values are not showing up
df_archive_clean.query('name in @invalid_names')

Empty DataFrame
Columns: [tweet_id, in_reply_to_status_id, in_reply_to_user_id, timestamp, source, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls, rating_numerator, rating_denominator, name, classification, text_clean, retweet_count, favorite_count, url]
Index: []

### Quality Issue 2
Timestamp and retweeted_status_timestamp is represented as an object and should be converted to a datetime.

##### Define
- Change the data type to datetime

##### Code

In [1237]:
#change the columns to datetime data type
df_archive_clean['timestamp']=pd.to_datetime(df_archive_clean['timestamp'])
df_archive_clean['retweeted_status_timestamp']=pd.to_datetime(df_archive_clean['retweeted_status_timestamp'])

##### Test

In [1238]:
#test the dataframe info to ensure data type was changed
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2340 entries, 0 to 2339
Data columns (total 17 columns):
tweet_id                      2340 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2340 non-null datetime64[ns]
source                        2340 non-null object
retweeted_status_id           167 non-null float64
retweeted_status_user_id      167 non-null float64
retweeted_status_timestamp    167 non-null datetime64[ns]
expanded_urls                 2281 non-null object
rating_numerator              2340 non-null int64
rating_denominator            2340 non-null int64
name                          1532 non-null object
classification                378 non-null category
text_clean                    2340 non-null object
retweet_count                 2340 non-null int64
favorite_count                2340 non-null int64
url                           2067 non-null object
dtypes: category(

### Quality Issue 3
Expanded URLs seem to consist of duplicate Twitter URLs and also URLs to other sites (like go fund me) all jumbled into a single attribute

##### Define
- Extract twitter URLs from the attribute and remove duplicates

##### Code

In [1239]:
#use regex to only extract the twitter URLs
df_archive_clean['expanded_urls']=df_archive_clean['expanded_urls'].str.extract('(https?://twitter(.+?),)')


In [1240]:
#remove unncessary comma at the end of expanded urls
df_archive_clean['expanded_urls']=df_archive_clean['expanded_urls'].str[:-1]

##### Test

In [1241]:
#verify the expanded URLs
df_archive_clean['expanded_urls'].value_counts()

https://twitter.com/dog_rates/status/789530877013393408/photo/1    2
https://twitter.com/dog_rates/status/774314403806253056/photo/1    2
https://twitter.com/dog_rates/status/822489057087389700/photo/1    2
https://twitter.com/dog_rates/status/860563773140209665/photo/1    2
https://twitter.com/dog_rates/status/782305867769217024/photo/1    2
https://twitter.com/dog_rates/status/820314633777061888/photo/1    2
https://twitter.com/dog_rates/status/863062471531167744/photo/1    2
https://twitter.com/dog_rates/status/829501995190984704/photo/1    2
https://twitter.com/dog_rates/status/833124694597443584/photo/1    2
https://twitter.com/dog_rates/status/819004803107983360/photo/1    2
https://twitter.com/dog_rates/status/679062614270468097/photo/1    2
https://twitter.com/dog_rates/status/782969140009107456/photo/1    2
https://twitter.com/dog_rates/status/687317306314240000/photo/1    2
https://twitter.com/dog_rates/status/866334964761202691/photo/1    2
https://twitter.com/dog_rates/stat

In [1242]:
#verify row counts
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2340 entries, 0 to 2339
Data columns (total 17 columns):
tweet_id                      2340 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2340 non-null datetime64[ns]
source                        2340 non-null object
retweeted_status_id           167 non-null float64
retweeted_status_user_id      167 non-null float64
retweeted_status_timestamp    167 non-null datetime64[ns]
expanded_urls                 603 non-null object
rating_numerator              2340 non-null int64
rating_denominator            2340 non-null int64
name                          1532 non-null object
classification                378 non-null category
text_clean                    2340 non-null object
retweet_count                 2340 non-null int64
favorite_count                2340 non-null int64
url                           2067 non-null object
dtypes: category(1

### Quality Issue 5
#### df_archive
- Small portion of rating denominators are not 10 and are inconsistent.

##### Define
- Update all denominators to 10

##### Code

In [1243]:
#update all denominators to 10
df_archive_clean['rating_denominator']=10

##### Test

In [1244]:
#verify all denominators are 10
df_archive_clean['rating_denominator'].value_counts()

10    2340
Name: rating_denominator, dtype: int64

### Quality Issue 6
#### df_archive_clean table
- Retweets seem like duplicate data in both archive and image_pred tables

##### Define
- remove the retweet data from df_archive and df_tweepy tables

##### Code

In [1245]:
#identify duplicate images
df_archive_clean.query('url=="https://t.co/Bb3xnpsWBC"')

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
934   752309394570878976                    NaN                  NaN   
1849  675354435921575936                    NaN                  NaN   

               timestamp  \
934  2016-07-11 01:11:51   
1849 2015-12-11 16:40:19   

                                                                                  source  \
934   <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>   
1849                  <a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>   

      retweeted_status_id  retweeted_status_user_id  \
934          6.753544e+17              4.196984e+09   
1849                  NaN                       NaN   

     retweeted_status_timestamp  \
934         2015-12-11 16:40:19   
1849                        NaT   

                                                        expanded_urls  \
934   https://twitter.com/dog_rates/status/675354435921575936/video/1   
1849                                                              NaN   

      rating_numerator  rating_denominator name classification  \
934                 13                  10  NaN            NaN   
1849                13                  10  NaN            NaN   

                                               text_clean  retweet_count  \
934   RT @dog_rates: Everyone needs to watch this. 13/10           17974   
1849                 Everyone needs to watch this. 13/10           17973   

      favorite_count                      url  
934                0  https://t.co/Bb3xnpsWBC  
1849           33710  https://t.co/Bb3xnpsWBC

In [1246]:
#store all of the retweeted tweet IDs as a copy in the retweets variable
retweets = df_archive_clean[~df_archive_clean['retweeted_status_id'].isna()]['tweet_id'].copy()

In [1247]:
#copy all of the duplicated tweets and store in variable deleted_tweets
deleted_tweets = df_image_pred_clean[df_image_pred_clean['jpg_url'].duplicated()].query('tweet_id in @retweets')['tweet_id'].copy()

In [1248]:
#drop duplicated tweets from df_image_pred table
df_image_pred_clean = df_image_pred_clean.drop(df_image_pred_clean.query('tweet_id in @deleted_tweets').index)

In [1249]:
#drop all of the duplicate tweet IDs from df_archive_clean table 
df_archive_clean = df_archive_clean.drop(df_archive_clean.query('tweet_id in @deleted_tweets').index)

##### Test

In [1250]:
#Check one of the duplicated =images
df_image_pred_clean.query('jpg_url =="https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg"')

tweet_id                                          jpg_url  \
591  679158373988876288  https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg   

     img_num prediction  confidence   dog  
591        1        pug    0.272205  True

In [1251]:
#check to see if the deleted tweets were removed
df_archive_clean.query('tweet_id in @deleted_tweets')

Empty DataFrame
Columns: [tweet_id, in_reply_to_status_id, in_reply_to_user_id, timestamp, source, retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp, expanded_urls, rating_numerator, rating_denominator, name, classification, text_clean, retweet_count, favorite_count, url]
Index: []

### Quality Issue 7
#### df_archive_clean table
- IDs should be integers instead of float.

##### Define
- Change the data type for the IDs to INT instead of float and replace all null values to 0.

##### Code

In [1252]:
#replace all of the NaN values for all of the ID fields with 0
df_archive_clean['retweeted_status_user_id'] = df_archive_clean['retweeted_status_user_id'].fillna(0)
df_archive_clean['retweeted_status_id'] = df_archive_clean['retweeted_status_id'].fillna(0)
df_archive_clean['in_reply_to_status_id'] = df_archive_clean['in_reply_to_status_id'].fillna(0)
df_archive_clean['in_reply_to_user_id'] = df_archive_clean['in_reply_to_user_id'].fillna(0)


In [1253]:
#change all of the ID fields using astype to integer
df_archive_clean['retweeted_status_user_id'] = df_archive_clean['retweeted_status_user_id'].astype(int)
df_archive_clean['retweeted_status_id'] = df_archive_clean['retweeted_status_id'].astype(int)
df_archive_clean['in_reply_to_status_id'] = df_archive_clean['in_reply_to_status_id'].astype(int)
df_archive_clean['in_reply_to_user_id'] = df_archive_clean['in_reply_to_user_id'].astype(int)


##### Test

In [1254]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2279 entries, 0 to 2339
Data columns (total 17 columns):
tweet_id                      2279 non-null int64
in_reply_to_status_id         2279 non-null int64
in_reply_to_user_id           2279 non-null int64
timestamp                     2279 non-null datetime64[ns]
source                        2279 non-null object
retweeted_status_id           2279 non-null int64
retweeted_status_user_id      2279 non-null int64
retweeted_status_timestamp    106 non-null datetime64[ns]
expanded_urls                 542 non-null object
rating_numerator              2279 non-null int64
rating_denominator            2279 non-null int64
name                          1487 non-null object
classification                367 non-null category
text_clean                    2279 non-null object
retweet_count                 2279 non-null int64
favorite_count                2279 non-null int64
url                           2006 non-null object
dtypes: category(1),

### Quality Issue 8
#### df_image_pred table
- Inconsistent punctuation of the dog types in the prediction column.

##### Define
- Change the punctuation to proper case for the prediction column

##### Code

In [1255]:
#replace all underscores with a space
df_image_pred_clean['prediction'] = df_image_pred_clean['prediction'].str.replace('_',' ')

In [1256]:
#use the title function to make all first letters in words uppercase
df_image_pred_clean['prediction']=df_image_pred_clean['prediction'].str.title()

##### Test

In [1257]:
df_image_pred_clean

tweet_id  \
0     666020888022790149   
1     666029285002620928   
2     666033412701032449   
3     666044226329800704   
4     666049248165822465   
5     666050758794694657   
6     666051853826850816   
7     666055525042405380   
8     666057090499244032   
9     666058600524156928   
10    666063827256086533   
11    666071193221509120   
12    666073100786774016   
13    666082916733198337   
14    666094000022159362   
15    666099513787052032   
16    666102155909144576   
17    666104133288665088   
18    666268910803644416   
19    666273097616637952   
20    666287406224695296   
21    666293911632134144   
22    666337882303524864   
23    666345417576210432   
24    666353288456101888   
25    666362758909284353   
26    666373753744588802   
27    666396247373291520   
28    666407126856765440   
29    666411507551481857   
...                  ...   
2045  886366144734445568   
2046  886680336477933568   
2047  886736880519319552   
2048  886983233522544640   
2049  887101392804085760   
2050  887343217045368832   
2051  887473957103951883   
2052  887517139158093824   
2053  887705289381826560   
2054  888078434458587136   
2055  888202515573088257   
2056  888554962724278272   
2057  888804989199671297   
2058  888917238123831296   
2059  889278841981685760   
2060  889531135344209921   
2061  889638837579907072   
2062  889665388333682689   
2063  889880896479866881   
2064  890006608113172480   
2065  890240255349198849   
2066  890609185150312448   
2067  890729181411237888   
2068  890971913173991426   
2069  891087950875897856   
2070  891327558926688256   
2071  891689557279858688   
2072  891815181378084864   
2073  892177421306343426   
2074  892420643555336193   

                                                                                      jpg_url  \
0                                             https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1                                             https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2                                             https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3                                             https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4                                             https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   
5                                             https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg   
6                                             https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg   
7                                             https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg   
8                                             https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg   
9                                             https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg   
10                                            https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg   
11                                            https://pbs.twimg.com/media/CT5cN_3WEAAlOoZ.jpg   
12                                            https://pbs.twimg.com/media/CT5d9DZXAAALcwe.jpg   
13                                            https://pbs.twimg.com/media/CT5m4VGWEAAtKc8.jpg   
14                                            https://pbs.twimg.com/media/CT5w9gUW4AAsBNN.jpg   
15                                            https://pbs.twimg.com/media/CT51-JJUEAA6hV8.jpg   
16                                            https://pbs.twimg.com/media/CT54YGiWUAEZnoK.jpg   
17                                            https://pbs.twimg.com/media/CT56LSZWoAAlJj2.jpg   
18                                            https://pbs.twimg.com/media/CT8QCd1WEAADXws.jpg   
19                                            https://pbs.twimg.com/media/CT8T1mtUwAA3aqm.jpg   
20                                            https://pbs.twimg.com/media/CT8g3BpUEAAuFjg.jpg   
21                                            https://pbs.twimg.com/media/CT8mx7KW4AEQu8N.jpg   
22                                            ht

In [1259]:
#save all of the cleaned dataframes to one master csv
twitter_archive_master = df_archive_clean.merge(df_image_pred_clean, on='tweet_id')
twitter_archive_master.to_csv('twitter_archive_master.csv', index=False)